In [8]:
import pandas as pd
from math import sqrt, radians, sin, cos, atan2, degrees
import folium
from folium import Map, PolyLine, Marker
from folium.plugins import MarkerCluster
from folium.utilities import validate_location
import tensorflow as tf

In [5]:
df = pd.read_csv('uni7_data.csv')
nodes = []
for idx, row in df.iterrows():
    nodes.append({
        'id': idx,
        'lat': row['latitude'],
        'lon': row['longitude'],
        'tof': row['timeOfFlight'],  
        'speed': row['speed'],   
        'bearing': row['bearing']   
    })

In [6]:
class AStarPathPlanner:
    def __init__(self, nodes):
        self.nodes = nodes
        self.graph = self._build_graph()
    
    def _haversine(self, node1, node2):
        """Calculate distance between two GPS points (in meters)"""
        R = 6371000  # Earth radius in meters
        lat1, lon1 = radians(node1['lat']), radians(node1['lon'])
        lat2, lon2 = radians(node2['lat']), radians(node2['lon'])
        
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        
        a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
        c = 2 * atan2(sqrt(a), sqrt(1-a))
        
        return R * c
    
    def _build_graph(self):
        """Create a connected graph from recorded path points"""
        graph = {}
        for i, node in enumerate(self.nodes):
            neighbors = []
            for j in range(i+1, min(i+4, len(self.nodes))):
                dist = self._haversine(node, self.nodes[j])
                penalty = 1000 if self.nodes[j]['tof'] < 1000 else 1
                neighbors.append((j, dist * penalty))
            graph[i] = neighbors
        return graph
    
    def heuristic(self, a, b):
        """Straight-line distance heuristic"""
        return self._haversine(self.nodes[a], self.nodes[b])
    
    def a_star(self, start_idx, goal_idx):
        """Find optimal path using A* algorithm"""
        from heapq import heappush, heappop
        
        open_set = []
        heappush(open_set, (0, start_idx))
        
        came_from = {}
        g_score = {node: float('inf') for node in range(len(self.nodes))}
        g_score[start_idx] = 0
        
        f_score = {node: float('inf') for node in range(len(self.nodes))}
        f_score[start_idx] = self.heuristic(start_idx, goal_idx)
        
        while open_set:
            _, current = heappop(open_set)
            
            if current == goal_idx:
                path = []
                while current in came_from:
                    path.append(current)
                    current = came_from[current]
                path.append(start_idx)
                return path[::-1]
            
            for neighbor, dist in self.graph.get(current, []):
                tentative_g = g_score[current] + dist
                
                if tentative_g < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g
                    f_score[neighbor] = tentative_g + self.heuristic(neighbor, goal_idx)
                    heappush(open_set, (f_score[neighbor], neighbor))
        
        return [] 

planner = AStarPathPlanner(nodes)
start_node = 0    
goal_node = 236  
optimal_path = planner.a_star(start_node, goal_node)


In [ ]:
bounds = [[df['latitude'].min(), df['longitude'].min()], 
          [df['latitude'].max(), df['longitude'].max()]]
map_center = [(bounds[0][0] + bounds[1][0])/2, (bounds[0][1] + bounds[1][1])/2]

zoomed_map = folium.Map(location=map_center, zoom_start=15)
zoomed_map.fit_bounds(bounds)

original_coords = df[['latitude', 'longitude']].values.tolist()
folium.PolyLine(
    locations=original_coords,
    color='blue',
    weight=2,
    opacity=0.5,
    tooltip="Original Path"
).add_to(zoomed_map)


optimal_path_coords = [original_coords[i] for i in optimal_path]  

folium.PolyLine(
    locations=optimal_path_coords,
    color='red',
    weight=4,
    opacity=0.9,
    tooltip="A* Optimized Path"
).add_to(zoomed_map)

folium.Marker(
    location=optimal_path_coords[0],
    icon=folium.Icon(color='green', icon='play'),
    tooltip="Start Point"
).add_to(zoomed_map)

folium.Marker(
    location=optimal_path_coords[-1],
    icon=folium.Icon(color='red', icon='stop'),
    tooltip="End Point"
).add_to(zoomed_map)


obstacles = df[df['timeOfFlight'] < 1000]
for _, row in obstacles.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=5,
        color='black',
        fill=True,
        fill_color='black',
        tooltip=f"Obstacle (TOF: {row['timeOfFlight']}mm)"
    ).add_to(zoomed_map)


marker_cluster = MarkerCluster().add_to(zoomed_map)
for _, row in df.iterrows():
    if row['direction'] == 'left':
        icon = folium.Icon(color='orange', icon='arrow-left')
    elif row['direction'] == 'right':
        icon = folium.Icon(color='orange', icon='arrow-right')
    elif row['direction'] == 'backward':
        icon = folium.Icon(color='purple', icon='arrow-down')
    else:
        icon = folium.Icon(color='blue', icon='arrow-up')
    
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        icon=icon,
        popup=f"""
            <b>Speed:</b> {row['speed']:.2f} m/s<br>
            <b>Direction:</b> {row['direction'] or 'forward'}<br>
            <b>TOF:</b> {row['timeOfFlight']} mm
        """
    ).add_to(marker_cluster)

zoomed_map